In [1]:
import os
import sys
import  time
import random
import numpy as np
from tqdm import tqdm
from PIL import Image
import tensorflow as tf
from skimage.io import imread
from keras import backend as K
import matplotlib.pyplot as plt
plt.style.use("ggplot")
from skimage.transform import resize
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Input
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import binary_crossentropy
from tensorflow.keras.preprocessing.image import  img_to_array
from tensorflow.keras.models import Model
from tensorflow.keras.layers import  BatchNormalization, Activation, Dropout
from tensorflow.keras.layers import Conv2D, Conv2DTranspose
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import concatenate
from TV_UNET import get_unet, TV_bin_loss
from tensorflow.keras.metrics import Recall, Precision
import os
import random
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
plt.style.use("ggplot")
from tqdm import tqdm_notebook, tnrange
from itertools import chain
from skimage.io import imread, imshow, concatenate_images
from skimage.transform import resize
from skimage.morphology import label
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Input, BatchNormalization, Activation, Dense, Dropout
from tensorflow.keras.layers import Lambda, RepeatVector, Reshape
from tensorflow.keras.layers import Conv2D, Conv2DTranspose
from tensorflow.keras.layers import MaxPooling2D, GlobalMaxPool2D
from tensorflow.keras.layers import concatenate, add
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import binary_crossentropy
from tensorflow.keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from tensorflow.keras import backend as K 
import h5py

In [2]:
################## TV-Unet

'''#########################################################################'''
im_width = 128
im_height = 128
EpoachesNo = 10
BatcheSZE = 32
num_class = 2

def conv2d_block(input_tensor, n_filters, kernel_size = 3, batchnorm = True):
    """Function to add 2 convolutional layers with the parameters passed to it"""
    # first layer
    x = Conv2D(filters = n_filters, kernel_size = (kernel_size, kernel_size),\
              kernel_initializer = 'he_normal', padding = 'same')(input_tensor)
    if batchnorm:
        x = BatchNormalization()(x)
    x = Activation('relu')(x)
    
    # second layer
    x = Conv2D(filters = n_filters, kernel_size = (kernel_size, kernel_size),\
              kernel_initializer = 'he_normal', padding = 'same')(x)
    if batchnorm:
        x = BatchNormalization()(x)
    x = Activation('relu')(x)
    
    return x

'''#########################################################################'''
def get_unet(input_img, n_filters = 64, dropout = 0.2, batchnorm = True):
    """Function to define the UNET Model"""
    # Contracting Path
    c1 = conv2d_block(input_img, n_filters * 1, kernel_size = 3, batchnorm = batchnorm)
    p1 = MaxPooling2D((2, 2))(c1)
    p1 = Dropout(dropout)(p1)
    
    c2 = conv2d_block(p1, n_filters * 2, kernel_size = 3, batchnorm = batchnorm)
    p2 = MaxPooling2D((2, 2))(c2)
    p2 = Dropout(dropout)(p2)
    
    c3 = conv2d_block(p2, n_filters * 4, kernel_size = 3, batchnorm = batchnorm)
    p3 = MaxPooling2D((2, 2))(c3)
    p3 = Dropout(dropout)(p3)
    
    c4 = conv2d_block(p3, n_filters * 8, kernel_size = 3, batchnorm = batchnorm)
    p4 = MaxPooling2D((2, 2))(c4)
    p4 = Dropout(dropout)(p4)
    
    c5 = conv2d_block(p4, n_filters = n_filters * 16, kernel_size = 3, batchnorm = batchnorm)
    
    # Expansive Path
    u6 = Conv2DTranspose(n_filters * 8, (3, 3), strides = (2, 2), padding = 'same')(c5)
    u6 = concatenate([u6, c4])
    u6 = Dropout(dropout)(u6)
    c6 = conv2d_block(u6, n_filters * 8, kernel_size = 3, batchnorm = batchnorm)
    
    u7 = Conv2DTranspose(n_filters * 4, (3, 3), strides = (2, 2), padding = 'same')(c6)
    u7 = concatenate([u7, c3])
    u7 = Dropout(dropout)(u7)
    c7 = conv2d_block(u7, n_filters * 4, kernel_size = 3, batchnorm = batchnorm)
    
    u8 = Conv2DTranspose(n_filters * 2, (3, 3), strides = (2, 2), padding = 'same')(c7)
    u8 = concatenate([u8, c2])
    u8 = Dropout(dropout)(u8)
    c8 = conv2d_block(u8, n_filters * 2, kernel_size = 3, batchnorm = batchnorm)
    
    u9 = Conv2DTranspose(n_filters * 1, (3, 3), strides = (2, 2), padding = 'same')(c8)
    u9 = concatenate([u9, c1])
    u9 = Dropout(dropout)(u9)
    c9 = conv2d_block(u9, n_filters * 1, kernel_size = 3, batchnorm = batchnorm)
    
    outputs = Conv2D(num_class, (1, 1), activation='sigmoid')(c9)
    model = Model(inputs=[input_img], outputs=[outputs])
    return model

'''#########################################################################'''
def TV_bin_loss(y_true, y_pred):
  y_true_f = K.flatten(y_true)
  y_pred_f = K.flatten(y_pred)

  bin_loss = binary_crossentropy(y_true_f,y_pred_f )
    
  images=y_pred[:,:,:,1]
  value = tf.reduce_mean(tf.abs(images[:,1:,:] - images[:,:-1,:])) + \
            tf.reduce_mean(tf.abs(images[:,:,1:] - images[:,:,:-1]))

  return 2.4e-7*value + bin_loss


def dice_coef(y_pred, y_true):
    
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + 0.0001) / (K.sum(y_true_f) + K.sum(y_pred_f) + 0.0001)


def dice_loss(y_true, y_pred):
    return 1-dice_coef(y_true, y_pred)

def my_loss(y_true, y_pred):
    layer_names=[layer.name for layer in model.layers]
    for l in layer_names:
        if l==layer_names[-1]:
            value = TV_bin_loss(y_true, y_pred)
        else:
            value = binary_crossentropy(K.flatten(y_true),K.flatten(y_pred))
    return value

In [3]:
############3 Collecct dataset

# read datasets

combined_data = h5py.File("h5_datasets/combined_CT_datasets.h5", "r")

print(combined_data.keys())

X_train = np.array(combined_data["X_train"])
X_valid = np.array(combined_data["X_valid"])
y_train = np.array(combined_data["y_train"])
y_valid = np.array(combined_data["y_valid"])
label_test = np.array(combined_data["label_test"])
y = np.array(combined_data["y"])

<KeysViewHDF5 ['X_train', 'X_valid', 'label_test', 'y', 'y_train', 'y_valid']>


In [4]:
############### Train model

input_img = Input((im_height, im_width,1), name='img')
model = get_unet(input_img, n_filters=64, dropout=0.2, batchnorm=True)
model.compile(optimizer=Adam(learning_rate=0.001) , loss = [my_loss], metrics=['accuracy',dice_loss,Recall(name='recall_1'),
                                                            Precision(name='pre_1')])

callbacks = [
    EarlyStopping(patience=50, verbose=1),
    ReduceLROnPlateau(factor=0.1, patience=5, min_lr=0.00001, verbose=1),
    ModelCheckpoint('model-TV-UNet1.h5', verbose=1, save_best_only=True, save_weights_only=True)
]

results = model.fit(X_train, y_train, batch_size=BatcheSZE, epochs=EpoachesNo, callbacks=callbacks, validation_data=(X_valid, y_valid))

2022-02-10 17:18:35.701085: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-02-10 17:18:35.710687: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-02-10 17:18:35.711578: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-02-10 17:18:35.712467: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

Epoch 1/10


2022-02-10 17:18:43.713015: I tensorflow/stream_executor/cuda/cuda_dnn.cc:366] Loaded cuDNN version 8201
2022-02-10 17:18:44.315012: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory


41/41 [==============================] - ETA: 0s - loss: 0.4261 - accuracy: 0.8527 - dice_loss: 0.3237 - recall_1: 0.8184 - pre_1: 0.8772
Epoch 00001: val_loss improved from inf to 11.89126, saving model to model-TV-UNet1.h5
41/41 [==============================] - 49s 810ms/step - loss: 0.4261 - accuracy: 0.8527 - dice_loss: 0.3237 - recall_1: 0.8184 - pre_1: 0.8772 - val_loss: 11.8913 - val_accuracy: 0.3895 - val_dice_loss: 0.6586 - val_recall_1: 0.4039 - val_pre_1: 0.2969 - lr: 0.0010
Epoch 2/10
41/41 [==============================] - ETA: 0s - loss: 0.2035 - accuracy: 0.9613 - dice_loss: 0.1593 - recall_1: 0.9611 - pre_1: 0.9610
Epoch 00002: val_loss improved from 11.89126 to 6.93412, saving model to model-TV-UNet1.h5
41/41 [==============================] - 22s 526ms/step - loss: 0.2035 - accuracy: 0.9613 - dice_loss: 0.1593 - recall_1: 0.9611 - pre_1: 0.9610 - val_loss: 6.9341 - val_accuracy: 0.4003 - val_dice_loss: 0.5834 - val_recall_1: 0.4704 - val_pre_1: 0.3925 - lr: 0.0010


In [ ]:
##############
